Yolov8로 해양 쓰레기 데이터를 학습해봅시다. 

https://docs.ultralytics.com/modes/train/

데이터셋

https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=506

In [19]:
!git clone https://github.com/ultralytics/ultralytics
!cd ultralytics
!pip install -e .

fatal: destination path 'ultralytics' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


학습데이터셋의 라벨 파일을 불러와봅시다

In [20]:
import json

file = '/content/drive/MyDrive/Teaching/DL 202301/해양쓰레기/라벨링데이터/TL_부유쓰레기(Bbox)/TL1/FD_1023.json'
with open(file, 'r') as f:  # r은 읽기 모드
  label = json.load(f)  #json 파일 읽기

In [21]:
label

{'version': '4.5.9',
 'flags': {},
 'shapes': [{'label': 'Plastic_Buoy',
   'points': [[474, 322], [540, 375]],
   'origin': 'fishing',
   'group_id': None,
   'shape_type': 'rectangle',
   'flags': {}},
  {'label': 'Plastic_Buoy',
   'points': [[692, 395], [821, 529]],
   'origin': 'fishing',
   'group_id': None,
   'shape_type': 'rectangle',
   'flags': {}},
  {'label': 'Rope',
   'points': [[618, 464], [871, 734]],
   'origin': 'fishing',
   'group_id': None,
   'shape_type': 'rectangle',
   'flags': {}},
  {'label': 'Net',
   'points': [[667, 476], [899, 710]],
   'origin': 'fishing',
   'group_id': None,
   'shape_type': 'rectangle',
   'flags': {}},
  {'label': 'Plastic_Buoy',
   'points': [[816, 378], [1025, 629]],
   'origin': 'fishing',
   'group_id': None,
   'shape_type': 'rectangle',
   'flags': {}},
  {'label': 'Rope',
   'points': [[842, 307], [896, 400]],
   'origin': 'fishing',
   'group_id': None,
   'shape_type': 'rectangle',
   'flags': {}}],
 'imagePath': 'FD_1023.j

yolov8 타입에 맞도록 변환해주어야 합니다. 

https://docs.ultralytics.com/datasets/detect/

```
<object-class> <x-center> <y-center> <width> <height>
```

* 0~1 사이로 표준화

* class 정보와 파일 경로는 yaml 파일에 저장

위 데이터셋은 class index가 없으므로 딕셔너리로 작성해주겠습니다

In [36]:
classes = ['Glass','Metal','Net','PET_Bottle','Plastic_Buoy', 'Plastic_Buoy_China','Plastic_ETC','Rope','Styrofoam_Box','Styrofoam_Buoy','Styrofoam_Piece']

In [37]:
class_names = {x: i for i, x in enumerate(classes)}

In [38]:
class_names

{'Glass': 0,
 'Metal': 1,
 'Net': 2,
 'PET_Bottle': 3,
 'Plastic_Buoy': 4,
 'Plastic_Buoy_China': 5,
 'Plastic_ETC': 6,
 'Rope': 7,
 'Styrofoam_Box': 8,
 'Styrofoam_Buoy': 9,
 'Styrofoam_Piece': 10}

필요정보를 추출해봅시다

In [25]:
img_width = label['imageWidth']
img_height = label['imageHeight']

In [26]:
new_labels = []

for i in label['shapes']:

  # 클래스 정보
  class_idx = class_names[i['label']]

  # 포인트 정보
  x_min, y_min = i['points'][0]
  x_max, y_max = i['points'][1]

  # 중심점 기준으로 변경
  x_center = (x_max+x_min)/2
  y_center = (y_max+y_min)/2
  width = x_max-x_min
  height = y_max-y_min

  #0~1 사이로 변경
  x_center /= img_width
  y_center /= img_height
  width /= img_width
  height /= img_height

  new_labels.append([class_idx, x_center, y_center, width, height])

In [27]:
new_labels

[[4, 0.4225, 0.435625, 0.055, 0.06625],
 [4, 0.6304166666666666, 0.5775, 0.1075, 0.1675],
 [7, 0.6204166666666666, 0.74875, 0.21083333333333334, 0.3375],
 [2, 0.6525, 0.74125, 0.19333333333333333, 0.2925],
 [4, 0.7670833333333333, 0.629375, 0.17416666666666666, 0.31375],
 [7, 0.7241666666666666, 0.441875, 0.045, 0.11625]]

저장할 폴더를 만들고 동일한 이름의 txt 파일로 저장해봅시다

In [28]:
import os

filename, ext = os.path.splitext(os.path.basename(file))
print(filename) 

FD_1023


In [29]:
path_save = '/content/drive/MyDrive/Teaching/DL 202301/해양쓰레기/라벨링데이터/TL_부유쓰레기(Bbox)/TL1_TXT/'
file_save = path_save + filename + '.txt'
with open(file_save, 'w') as f:
    for inner_list in new_labels:
        line = ' '.join(str(elem) for elem in inner_list)
        f.write(line + '\n')

이제 함수로 만들고 전체 라벨을 변환해보겠습니다. 

In [30]:
def to_yoloformat(file, path_save):
  with open(file, 'r') as f:  # r은 읽기 모드
    label = json.load(f)  #json 파일 읽기

  # 이미지 사이즈 추출
  img_width = label['imageWidth']
  img_height = label['imageHeight']

  # 저장할 파일 만들기
  filename, ext = os.path.splitext(os.path.basename(file))
  file_save = path_save + filename + '.txt'
  with open(file_save, 'w') as f:

    for i in label['shapes']:

      # 클래스 정보
      class_idx = class_names[i['label']]

      # 포인트 정보
      x_min, y_min = i['points'][0]
      x_max, y_max = i['points'][1]

      # 중심점 기준으로 변경
      x_center = (x_max+x_min)/2
      y_center = (y_max+y_min)/2
      width = x_max-x_min
      height = y_max-y_min

      #0~1 사이로 변경
      x_center /= img_width
      y_center /= img_height
      width /= img_width
      height /= img_height

      inner_list = [class_idx, x_center, y_center, width, height]

      line = ' '.join(str(elem) for elem in inner_list)
      f.write(line + '\n')

      

    

이제 폴더를 읽어서 변환합시다

In [39]:
folder_path = "/content/drive/MyDrive/Teaching/DL 202301/해양쓰레기/라벨링데이터/TL_부유쓰레기(Bbox)/TL1/"

for item_name in os.listdir(folder_path):
    file = os.path.join(folder_path, item_name)
    if os.path.isfile(file):  # 파일인 경우에만 출력
        to_yoloformat(file, path_save)